<a href="https://colab.research.google.com/github/RogerHeederer/ML_practice_MyFirstML_B/blob/master/LSTM_ParagraphClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

source reference : https://github.com/wikibook/machine-learnin

위 레퍼런스를 참조하여 공부하였으며, 필요하다고 생각되는 부분에 추가적인 설명 ,코드 추가, 그림 삽입 등을 덧붙였습니다.

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

In [3]:

from IPython.display import Image

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

# 항상 같은 결과를 갖기 위해 랜덤 시드 설정
tf.random.set_seed(1)
np.random.seed(1)

In [4]:
paragraph_dict_list = [
         {'paragraph': 'dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
         {'paragraph': 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
         {'paragraph': 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
         {'paragraph': 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
         {'paragraph': 'the biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
         {'paragraph': 'the garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
         {'paragraph': 'our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
         {'paragraph': 'what i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
         {'paragraph': 'the drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
         {'paragraph': 'despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
    
         {'paragraph': 'the four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
         {'paragraph': 'the briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
         {'paragraph': 'stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
         {'paragraph': 'when it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
         {'paragraph': 'the team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
         {'paragraph': 'the perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
         {'paragraph': 'liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
         {'paragraph': 'alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
         {'paragraph': 'but the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
         {'paragraph': 'then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the third set there were exhilarating rallies with both chasing to the net both retrieving what looked like winning shots nadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]
df = pd.DataFrame(paragraph_dict_list)
df = df[['paragraph', 'category']]

In [5]:
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


In [6]:
df.tail()

,paragraph,category
15,the perseyside outfit finished in fourth place...,sports
16,liverpool fc will return to premier league act...,sports
17,alisson signed for liverpool fc from as roma t...,sports
18,but the rankings during that run-in to new yor...,sports
19,then came the oh-so-familiar djokovic-nadal no...,sports


In [9]:
#데이터 전처리
def get_vocab_size(df):
  results = set() # 단어 중복 제거
  df['paragraph'].str.lower().str.split().apply(results.update)
  return len(results)

In [10]:
# 전체 단어 갯수
vocab_size = get_vocab_size(df)

In [11]:
vocab_size

536

In [15]:
paragraphs = df['paragraph'].tolist()
encoded_paragraphs = [one_hot(paragraph, vocab_size) for paragraph in paragraphs]

In [16]:
paragraphs[0:5]

['dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait',
 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations',
 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table',
 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition',
 'the biscuits and gravy was too salty two people in my group had the gravy and all thoug

In [29]:
print(encoded_paragraphs[0])

[109, 239, 321, 518, 382, 136, 207, 239, 92, 204, 212, 254, 258, 434, 440, 13, 469, 430, 340, 124, 415, 354, 408, 150, 228, 394, 509, 273, 430, 4, 435, 252, 298, 259, 321, 8, 384, 193, 211, 175, 346, 509, 111, 336, 260, 492, 306, 87, 215, 212, 37, 430, 313]


In [22]:
def get_max_length(df):
  max_length = 0
  for row in df['paragraph']:
    if len(row.split(" ")) > max_length:
      max_length = len(row.split(" "))
  return max_length

max_length = get_max_length(df)
print(max_length)

91


In [24]:
#문장마다 단어 갯수가 다르기 때문에, 제로 패딩으로 문장의 길이를 동일하게 만듬
padded_paragraphs_encoding = pad_sequences(encoded_paragraphs, maxlen=max_length, padding='post')

In [70]:
padded_paragraphs_encoding, padded_paragraphs_encoding.shape

(array([[109, 239, 321, ...,   0,   0,   0],
        [243, 440,  13, ...,   0,   0,   0],
        [506,  94, 322, ...,   0,   0,   0],
        ...,
        [ 19, 382, 252, ...,   0,   0,   0],
        [258, 212, 472, ...,   0,   0,   0],
        [278, 273, 212, ...,   0,   0,   0]], dtype=int32), (20, 91))

In [26]:
categories = df['category'].tolist()
categories

['food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'sports',
 'sports',
 'sports',
 'sports',
 'sports',
 'sports',
 'sports',
 'sports',
 'sports',
 'sports']

In [27]:
# 카데고리 또한 수치로 변형시켜줘야 한다
def category_encode(category):
  if category == 'food':
    return [1,0]
  else:
    return [0,1]

encoded_category = [category_encode(category) for category in categories]
print(encoded_category)

[[1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1]]


주제 분류하는 모델 구현

In [30]:

Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/lstm_model_overview.png", width=500, height=250)

In [59]:
model = Sequential()
model.add(Embedding(vocab_size, 5, input_length=max_length))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [60]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [72]:
model.layers[0].weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(536, 5) dtype=float32, numpy=
 array([[-0.01021043,  0.03155854,  0.01192266, -0.02288668,  0.00437133],
        [ 0.04962002, -0.00266556, -0.06787688, -0.01795626,  0.07024096],
        [ 0.04736006,  0.04116863,  0.00529343,  0.0270474 , -0.00992645],
        ...,
        [ 0.06880832, -0.01124398, -0.01989429, -0.01539994,  0.04485341],
        [ 0.03252177,  0.05253371, -0.02468635,  0.02124182,  0.03354437],
        [ 0.00122338,  0.00200092, -0.02889839,  0.01319774,  0.05843382]],
       dtype=float32)>]

In [61]:
train_X = np.array(padded_paragraphs_encoding)
train_Y = np.array(encoded_category)

In [62]:
train_X[0:2]

array([[109, 239, 321, 518, 382, 136, 207, 239,  92, 204, 212, 254, 258,
        434, 440,  13, 469, 430, 340, 124, 415, 354, 408, 150, 228, 394,
        509, 273, 430,   4, 435, 252, 298, 259, 321,   8, 384, 193, 211,
        175, 346, 509, 111, 336, 260, 492, 306,  87, 215, 212,  37, 430,
        313,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [243, 440,  13, 150, 124, 175, 408, 258, 302,  15, 146, 364, 394,
        444, 137, 199, 408, 284, 193,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   

In [63]:
train_Y[0:2]

array([[1, 0],
       [1, 0]])

In [65]:
print('Train...')
model.fit(train_X, train_Y,batch_size=10,epochs=50)

Train...
Epoch 1/50
2/2 [==============================] - 0s 16ms/step - loss: 0.6941 - accuracy: 0.5500
Epoch 2/50
2/2 [==============================] - 0s 13ms/step - loss: 0.6927 - accuracy: 0.5000
Epoch 3/50
2/2 [==============================] - 0s 17ms/step - loss: 0.6927 - accuracy: 0.5000
Epoch 4/50
2/2 [==============================] - 0s 16ms/step - loss: 0.6926 - accuracy: 0.5000
Epoch 5/50
2/2 [==============================] - 0s 14ms/step - loss: 0.6924 - accuracy: 0.5500
Epoch 6/50
2/2 [==============================] - 0s 14ms/step - loss: 0.6922 - accuracy: 0.5500
Epoch 7/50
2/2 [==============================] - 0s 14ms/step - loss: 0.6919 - accuracy: 0.5500
Epoch 8/50
2/2 [==============================] - 0s 13ms/step - loss: 0.6917 - accuracy: 0.5500
Epoch 9/50
2/2 [==============================] - 0s 15ms/step - loss: 0.6914 - accuracy: 0.5500
Epoch 10/50
2/2 [==============================] - 0s 17ms/step - loss: 0.6912 - accuracy: 0.5500
Epoch 11/50
2/2 [===

In [67]:
score, acc = model.evaluate(train_X, train_Y, verbose=2)

1/1 - 0s - loss: 0.1676 - accuracy: 0.9500


In [68]:
print('test score: ', score)

test score:  0.16756096482276917


In [74]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 91, 5)             2680      
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                17920     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 22,746
Trainable params: 22,746
Non-trainable params: 0
_________________________________________________________________


In [77]:
model.layers[0]

In [90]:
model.layers[0].weights[0]

<tf.Variable 'embedding_1/embeddings:0' shape=(536, 5) dtype=float32, numpy=
array([[-0.01021043,  0.03155854,  0.01192266, -0.02288668,  0.00437133],
       [ 0.04962002, -0.00266556, -0.06787688, -0.01795626,  0.07024096],
       [ 0.04736006,  0.04116863,  0.00529343,  0.0270474 , -0.00992645],
       ...,
       [ 0.06880832, -0.01124398, -0.01989429, -0.01539994,  0.04485341],
       [ 0.03252177,  0.05253371, -0.02468635,  0.02124182,  0.03354437],
       [ 0.00122338,  0.00200092, -0.02889839,  0.01319774,  0.05843382]],
      dtype=float32)>

In [91]:
model.layers[1].weights[2]

<tf.Variable 'lstm_1/lstm_cell_1/bias:0' shape=(256,) dtype=float32, numpy=
array([ 2.44747810e-02,  3.15554179e-02,  3.43391635e-02,  1.61706954e-02,
        1.95455477e-02,  8.22307658e-04,  4.69247140e-02,  2.34648846e-02,
        1.99181736e-02,  1.55924205e-02,  2.05902066e-02,  1.16462335e-02,
        3.27920169e-03,  6.92736357e-03,  2.79465057e-02,  4.43815254e-03,
       -8.07309337e-03,  3.32085975e-02,  3.38948481e-02,  2.74020787e-02,
        3.12721878e-02,  2.83305589e-02,  8.46717041e-03,  3.22306603e-02,
        3.16473171e-02,  3.07868347e-02,  3.17280255e-02,  2.24202406e-02,
       -5.18579083e-03,  2.56428476e-02,  3.24185789e-02,  2.40560733e-02,
        2.91861836e-02,  1.85354855e-02,  3.13460156e-02,  4.79961513e-03,
        4.48783301e-02,  1.46639943e-02,  5.27493795e-03,  2.06000898e-02,
       -5.61239198e-03,  3.02170515e-02,  2.88858637e-02,  1.42548326e-02,
        2.96834875e-02,  3.06599196e-02,  4.01383685e-03,  2.23294739e-02,
        4.00861986e-02, 